# Robot Trading API in Python

### 1. Import Library Request to make http requests

In [1]:
#!pip install requests

import requests as rq

urlB = 'https://api.pro.coinbase.com'

### 2. Function wich convert the requests reponse (bytes) in JSON

In [2]:
import json

def convertResponse(res):
    res = res.decode("utf-8")
    resFinal = json.loads(res)
    return resFinal

### 3. Get a listof all available cryptocurrencies

In [11]:
reqList = rq.request('GET', urlB+'/currencies')

reqList = convertResponse(reqList.content)

print("Il y'a "+str(len(reqList))+" monnaies")

for el in reqList:
    print(el['id'])

Il y'a 30 monnaies
ALGO
DASH
OXT
ATOM
XLM
XRP
EOS
ZRX
BAT
LOOM
CVC
DNT
MANA
GNT
REP
LINK
BTC
EUR
LTC
GBP
USD
ETH
BCH
ETC
USDC
ZEC
XTZ
DAI
ZIL
MKR


### 4. Bid and Ask price for an asset

In [12]:
#check all the assets
reqList = rq.request('GET', urlB+'/products')

reqList = convertResponse(reqList.content)

print("Il y'a "+str(len(reqList))+" assets")

for el in reqList:
    print(el['id'])
    
print("")


#Functions getDepth
def getDepth(direction='ask', pair='BTC-USD'):
    reqList = rq.request('GET', urlB + '/products/' + pair + '/ticker')
    reqList = convertResponse(reqList.content)
    if(direction == 'ask'):
        print("Voici le '"+ direction +"' prix pour l'asset "+ pair +" : " + str(reqList['ask']))
    elif(direction == 'bid'):
        print("Voici le '"+ direction +"' prix pour l'asset "+ pair +" : " + str(reqList['bid']))
    else:
        print("Mauvaise direction")

getDepth()
print("")
getDepth('bid', 'BCH-EUR')
print("")
getDepth("Monnaies numériques c'est trop bien") #little joke hahah

Il y'a 56 assets
MANA-USDC
ZRX-USD
BAT-ETH
BCH-EUR
ETH-USDC
REP-USD
XLM-USD
XRP-BTC
REP-BTC
BCH-BTC
BTC-GBP
ETH-BTC
LTC-GBP
GNT-USDC
BAT-USDC
LTC-EUR
EOS-BTC
ETH-USD
BTC-USD
DASH-BTC
BTC-EUR
ETC-BTC
ETH-DAI
ZRX-BTC
BTC-USDC
XTZ-USD
XRP-EUR
EOS-EUR
LINK-ETH
ZEC-BTC
XRP-USD
EOS-USD
XLM-BTC
LINK-USD
ETC-GBP
BCH-USD
ATOM-USD
LOOM-USDC
CVC-USDC
ETC-EUR
ETC-USD
ATOM-BTC
DAI-USDC
LTC-BTC
XLM-EUR
ZRX-EUR
BCH-GBP
DASH-USD
ETH-GBP
OXT-USD
DNT-USDC
ZEC-USDC
LTC-USD
ETH-EUR
XTZ-BTC
ALGO-USD

Voici le 'ask' prix pour l'asset BTC-USD : 8657.45

Voici le 'bid' prix pour l'asset BCH-EUR : 311.48

Mauvaise direction


### 5. Order book for an asset

In [13]:
#Functions getBook
def getBook(pair='BTC-USD'):
    reqList = rq.request('GET', urlB + '/products/' + pair + '/book')
    reqList = convertResponse(reqList.content)
    print("Voici l'order book de la paire "+pair+" : "+str(reqList))

getBook()
print("")
getBook('BCH-EUR')

Voici l'order book de la paire BTC-USD : {'sequence': 11841623561, 'bids': [['8657.44', '5.46971051', 7]], 'asks': [['8657.45', '1.30319144', 1]]}

Voici l'order book de la paire BCH-EUR : {'sequence': 2828708043, 'bids': [['311.25', '6.43', 2]], 'asks': [['312.29', '2.66079156', 1]]}


### 6. Get Candles of an asset

In [14]:
import time

listAcceptedDuration = ['1m','5m','15m','1h','6h','1d']
listReqDuation = [60, 300, 900, 3600, 21600, 86400]

def refreshDataCandle(pair = 'BTC-USD', duration = '5m'):
    
    if(duration in listAcceptedDuration):
        dicRequest = dict({'granularity':listReqDuation[listAcceptedDuration.index(duration)]})
    
        reqCandles = rq.request('GET', urlB + '/products/' + pair + '/candles', params=dicRequest)
        reqCandles = convertResponse(reqCandles.content)
        print("Voici un exemple pour la paire : "+pair+", Date :"+time.ctime(reqCandles[0][0])[10:16]+", Low : "+str(reqCandles[0][1])+" , High : "+str(reqCandles[0][2])+" , Open : "+str(reqCandles[0][3])+" , Close : "+str(reqCandles[0][4])+".")
    else:
        print("La durée doit être dans la liste")
    
refreshDataCandle()
    
refreshDataCandle('BCH-EUR','6h')

Voici un exemple pour la paire : BTC-USD, Date : 16:55, Low : 8659.99 , High : 8664.92 , Open : 8660 , Close : 8662.2.
Voici un exemple pour la paire : BCH-EUR, Date : 13:00, Low : 291.92 , High : 313.64 , Open : 296.35 , Close : 311.03.


### 7. Connection to SQLite

In [4]:
import sqlite3

try:
    sqliteConnection = sqlite3.connect('Trading.db')
    cursor = sqliteConnection.cursor()
    print("Database created and Successfully Connected to SQLite")

    sqlite_select_Query = "select sqlite_version();"
    cursor.execute(sqlite_select_Query)
    record = cursor.fetchall()
    print("SQLite Database Version is: ", record)
    cursor.close()

except sqlite3.Error as error:
    print("Error while connecting to sqlite", error)
finally:
    if (sqliteConnection):
        sqliteConnection.close()
        print("The SQLite connection is closed")

Database created and Successfully Connected to SQLite
SQLite Database Version is:  [('3.29.0',)]
The SQLite connection is closed


### 8. Recreate the function to store candle in DB

In [9]:
import time
import sqlite3

listAcceptedDuration = ['1m','5m','15m','1h','6h','1d']
listReqDuation = [60, 300, 900, 3600, 21600, 86400]

def createTable(tableName): 
    try:
        sqliteConnection = sqlite3.connect('Trading.db')
        sqlite_Query = '''CREATE TABLE IF NOT EXISTS ''' + tableName + ''' (Id INTEGER PRIMARY KEY AUTOINCREMENT, date INT, high REAL, low REAL, open REAL, close REAL, volume REAL)'''
        cursor = sqliteConnection.cursor()
        cursor.execute(sqlite_Query)
        sqliteConnection.commit()
        cursor.close()
    except sqlite3.Error as error:
        print("Error : ", error)
    finally:
        if (sqliteConnection):
            sqliteConnection.close()
            
def dropTable(tableName): 
    try:
        sqliteConnection = sqlite3.connect('Trading.db')
        sqlite_Query = '''DROP TABLE ''' + tableName
        cursor = sqliteConnection.cursor()
        cursor.execute(sqlite_Query)
        sqliteConnection.commit()
        cursor.close()
    except sqlite3.Error as error:
        print("Error : ", error)
    finally:
        if (sqliteConnection):
            sqliteConnection.close()
            
def fillDB(tableName, data):
    try:
        values = str(data[0][0])+","+str(data[0][2])+","+str(data[0][1])+","+str(data[0][3])+","+str(data[0][4])+","+str(data[0][5])
        sqliteConnection = sqlite3.connect('Trading.db')
        sqlite_Query = '''INSERT INTO ''' + tableName + ''' (date, high, low, open, close, volume) VALUES(''' + values + ''')'''
        cursor = sqliteConnection.cursor()
        cursor.execute(sqlite_Query)
        sqliteConnection.commit()
        cursor.close()
    except sqlite3.Error as error:
        print("Error : ", error)
    finally:
        if (sqliteConnection):
            sqliteConnection.close()
    

def refreshDataCandle(pair = 'BTC-USD', duration = '5m'):
    
    if(duration in listAcceptedDuration):
        
        setTableName = str(pair.replace("-","_") + "_" + duration)
        
        createTable(setTableName)
        
        dicRequest = dict({'granularity':listReqDuation[listAcceptedDuration.index(duration)]})
    
        reqCandles = rq.request('GET', urlB + '/products/' + pair + '/candles', params=dicRequest)
        reqCandles = convertResponse(reqCandles.content)
        
        fillDB(setTableName, reqCandles)
        
        
    else:
        print("La durée doit être dans la liste")
    
refreshDataCandle()

refreshDataCandle('BCH-EUR','6h')

### 9. Check Data in DB

In [10]:
import time
import sqlite3

def selectDataDB(tableName):
    try:
        sqliteConnection = sqlite3.connect('Trading.db')
        cursor = sqliteConnection.cursor()

        sqlite_select_Query = "SELECT * FROM "+ tableName
        cursor.execute(sqlite_select_Query)
        record = cursor.fetchall()
        print(record)
        cursor.close()

    except sqlite3.Error as error:
        print("Error : ", error)
    finally:
        if (sqliteConnection):
            sqliteConnection.close()
    
selectDataDB("BTC_USD_5m")

selectDataDB("BCH_EUR_6h")

selectDataDB("LOLOLO")

[(1, 1579535400, 8642.73, 8642.72, 8642.73, 8642.72, 3.15617465), (2, 1579535700, 8664.92, 8659.99, 8660.0, 8662.2, 6.51654265)]
[(1, 1579521600, 313.64, 291.92, 296.35, 311.03, 1902.67699511)]
Error :  no such table: LOLOLO
